<a href="https://colab.research.google.com/github/bhernandeza5/Challenge_TelecomX/blob/main/challenge_telecom_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telecom parte 2 Latam



## Subir el archivo tratado en la parte 1

In [4]:
import pandas as pd

# Subir archivo manualmente
from google.colab import files
uploaded = files.upload()

# Leer el CSV
df = pd.read_csv("telecom_tratado.csv")
df.head()


Saving telecom_tratado.csv to telecom_tratado.csv


,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Cuentas_Diarias,Servicios_Activos
0,0002-ORFBO,0,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30,2.186667,5
1,0003-MKNFE,0,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40,1.996667,3
2,0004-TLHLJ,1,Male,0,No,No,4,Yes,No,Fiber optic,...,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85,2.463333,3
3,0011-IGKFF,1,Male,1,Yes,No,13,Yes,No,Fiber optic,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85,3.266667,6
4,0013-EXCHZ,1,Female,1,Yes,No,3,Yes,No,Fiber optic,...,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40,2.796667,4


## Eliminar columnas que no aportan valor

In [5]:
# Eliminar columnas irrelevantes
df = df.drop(columns=["customerID"])

print("✅ Columnas eliminadas: customerID")
print("Columnas actuales:", df.columns.tolist())


✅ Columnas eliminadas: customerID
Columnas actuales: ['Churn', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly', 'Charges.Total', 'Cuentas_Diarias', 'Servicios_Activos']


In [6]:
# Separar variables categóricas y numéricas
cat_cols = df.select_dtypes(include="object").columns.tolist()
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Categorical columns:", cat_cols)
print("Numeric columns:", num_cols)


Categorical columns: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']
Numeric columns: ['Churn', 'SeniorCitizen', 'tenure', 'Charges.Monthly', 'Charges.Total', 'Cuentas_Diarias', 'Servicios_Activos']


## One Hot Encoder

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Columnas categóricas
cat_cols = df.select_dtypes(include="object").columns.tolist()

# Separar target
y = df["Churn"]
X = df.drop(columns=["Churn"])

# Definir transformador
ct = ColumnTransformer(
    transformers=[("encoder", OneHotEncoder(drop="first"), cat_cols)],
    remainder="passthrough"
)

X_encoded = ct.fit_transform(X)

print("Shape transformado:", X_encoded.shape)


Shape transformado: (7043, 32)


## Proporción de clases en Churn

In [8]:
# Distribución de la variable objetivo
churn_counts = y.value_counts()
churn_proportion = y.value_counts(normalize=True) * 100

print("Conteo de clientes por clase:")
print(churn_counts)
print("\nProporción (%) de clientes por clase:")
print(churn_proportion)


Conteo de clientes por clase:
Churn
0    5174
1    1869
Name: count, dtype: int64

Proporción (%) de clientes por clase:
Churn
0    73.463013
1    26.536987
Name: proportion, dtype: float64


### Resultado:

* 5174 clientes (73.5%) permanecieron en la empresa.

* 1869 clientes (26.5%) cancelaron.

Existe un desbalance moderado: ~3 de cada 4 clientes permanecen, y 1 de cada 4 cancela.

Un modelo “ingenuo” que siempre prediga “no canceló” tendría 73% de accuracy, pero no detectaría a ningún cliente que sí cancela (recall=0).

Por eso, **accuracy** no es suficiente como métrica en este caso.

Habrá que enfocarse en **recall, precision, F1-score y ROC-AUC.**